<a href="https://colab.research.google.com/github/pam-lab/JupyterFiles/blob/main/Similarity_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading and processing

In [ ]:
!git clone https://github.com/language-ml/course-nlp-ir-1-text-exploring

Cloning into 'course-nlp-ir-1-text-exploring'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 401 (delta 2), reused 0 (delta 0), pack-reused 397
Receiving objects: 100% (401/401), 98.78 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Checking out files: 100% (245/245), done.


In [ ]:
!pip install -Uq camel_tools
import random 
import nltk
import pandas as pd
import codecs
import tqdm
import random 
import nltk
from pathlib import Path
import re
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_alef_bw
from camel_tools.utils.normalize import normalize_alef_hsb
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.utils import normalize
import _pickle as pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

quranic_dir = "/content/course-nlp-ir-1-text-exploring/exploring-datasets/religious_text"
df_quran = pd.read_csv(f'{quranic_dir}/quranic_data/id_text_with_orthographies.txt', sep='\t', header=None)
df_nahj = pd.read_csv(f'{quranic_dir}/nahj-al-balaqa/Nahj Al-Balaqa.txt', sep='\t',header=None)
verse_complete_dict = pd.Series(df_quran[1].tolist(), index=df_quran[0]).to_dict()
nahj_complete_dict = pd.Series(df_nahj[1].tolist(), index=df_nahj[0]).to_dict()

     |████████████████████████████████| 114 kB 7.7 MB/s 
     |████████████████████████████████| 418 kB 45.9 MB/s 
     |████████████████████████████████| 4.2 MB 44.7 MB/s 
     |████████████████████████████████| 175 kB 60.1 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 6.6 MB 10.0 MB/s 


In [ ]:
sahife_text=Path(f'{quranic_dir}/Saheefa/sahife_sajjadieh.txt').read_text().split('\n')
sahife_complete_dict = [re.sub('[(][۰-۹]+[)]','', item) for item in sahife_text if item.startswith('(')]

In [ ]:
sahife_complete_dict[12]

' حَمْداً یَرْتَفِعُ مِنَّا إِلَی أَعْلَی عِلِّیِّینَ فِي کِتَابٍ مَرْقُومٍ یَشْهَدُه\u200cُ الْمُقَرَّبُونَ. '

In [ ]:
nahj_complete_dict['2##186']

'وَ قَالَ ع لِلظّالِمِ الباَدیِ غَداً بِکَفّهِ عَضّةٌ'

In [ ]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

### Normalize

In [ ]:
def normalize_arabic(sentence):

    # Normalize alef variants to 'ا'
    sent_norm = normalize_unicode(sentence)
    
    sent_norm = normalize_alef_bw(sent_norm)
    # Normalize alef variants to 'ا'
    sent_norm = normalize_alef_ar(sentence)

    # Normalize alef maksura 'ى' to yeh 'ي'
    sent_norm = normalize_alef_maksura_ar(sent_norm)

    # Normalize teh marbuta 'ة' to heh 'ه'
    sent_norm = normalize_teh_marbuta_ar(sent_norm)
    return dediac_ar(sent_norm)

In [ ]:
verse_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}
nahj_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(nahj_complete_dict.items())}
sahife_complete_dict_nrmlz = [normalize_arabic(item) for item in tqdm.tqdm(sahife_complete_dict)]

100%|██████████| 924/924 [00:00<00:00, 6900.26it/s]


In [ ]:
print(verse_complete_dict['2##186'])
print(verse_complete_dict_nrmlz['2##186'])

print(nahj_complete_dict['2##186'])
print(nahj_complete_dict_nrmlz['2##186'])

print(sahife_complete_dict[34])
print(sahife_complete_dict_nrmlz[34])

وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ
واذا سالك عبادي عني فاني قريب ۖ اجيب دعوه الداع اذا دعان ۖ فليستجيبوا لي وليؤمنوا بي لعلهم يرشدون
وَ قَالَ ع لِلظّالِمِ الباَدیِ غَداً بِکَفّهِ عَضّةٌ
و قال ع للظالم البادی غدا بکفه عضه
 وَ کَاشَفَ فِي الدُّعَاءِ إِلَیْکَ حَامَّتَه‌ُ 
 و کاشف في الدعاء الیک حامته‌ 


In [ ]:
quranic_tokenized = [sents.split() for sents in tqdm.tqdm(verse_complete_dict_nrmlz.values())]
nahj_tokenized = [sents.split() for sents in tqdm.tqdm(nahj_complete_dict_nrmlz.values())]
sahife_tokenized = [sents.split() for sents in tqdm.tqdm(sahife_complete_dict_nrmlz)]

100%|██████████| 924/924 [00:00<00:00, 142829.55it/s]


In [ ]:
quranic_tokenized[43]

['فتلقي',
 'ادم',
 'من',
 'ربه',
 'كلمات',
 'فتاب',
 'عليه',
 'ۚ',
 'انه',
 'هو',
 'التواب',
 'الرحيم']

In [ ]:
nahj_tokenized[43]

['قبح',
 'الله',
 'مصقله',
 'فعل',
 'فعل',
 'الساده',
 'و',
 'فر',
 'فرار',
 'العبید',
 'فما',
 'انطق',
 'مادحه',
 'حتی',
 'اسکته',
 'و',
 'لا',
 'صدق',
 'واصفه',
 'حتی',
 'بکته',
 'و',
 'لو',
 'اقام',
 'لاخذنا',
 'میسوره',
 'و',
 'انتظرنا',
 'بماله',
 'وفوره']

In [ ]:
sahife_tokenized[34]

['و', 'کاشف', 'في', 'الدعاء', 'الیک', 'حامته\u200c']

# Part 1 (IF-IDF)

In [ ]:
verse_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}
nahj_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(nahj_complete_dict.items())}
sahife_dict_nrmlz = {normalize_arabic(v) for v in tqdm.tqdm(sahife_complete_dict)}


100%|██████████| 6236/6236 [00:00<00:00, 13148.05it/s]

100%|██████████| 800/800 [00:00<00:00, 2781.20it/s]

100%|██████████| 924/924 [00:00<00:00, 5901.93it/s]


In [ ]:
df_verse_complete_dict = pd.DataFrame(list(verse_complete_dict.items()), columns = ['num','verse'])
df_verse_complete_dict_nrmlz = pd.DataFrame(list(verse_dict_nrmlz.items()), columns = ['num','verse'])
df_nahj_complete_dict = pd.DataFrame(list(nahj_complete_dict.items()), columns = ['num','verse'])
df_nahj_complete_dict_nrmlz = pd.DataFrame(list(nahj_dict_nrmlz.items()), columns = ['num','verse'])
df_sahife_complete_dict = pd.DataFrame(list(sahife_complete_dict), columns = ['verse'])
df_sahife_complete_dict_nrmlz = pd.DataFrame(list(sahife_dict_nrmlz), columns= ['verse'])

In [ ]:
# Create TfidfVectorizer object
vectorizer_quran = TfidfVectorizer(lowercase = False)
vectorizer_nahj = TfidfVectorizer(lowercase = False)
vectorizer_sahife = TfidfVectorizer(lowercase = False)
vectorizer_quran_char = TfidfVectorizer(lowercase = False,analyzer='char')
vectorizer_nahj_char = TfidfVectorizer(lowercase = False,analyzer='char')
vectorizer_sahife_char = TfidfVectorizer(lowercase = False,analyzer='char')

In [ ]:
# Generate matrix of word vectors
tfidf_matrix_quran = vectorizer_quran.fit_transform(df_verse_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_quran.shape)

(6236, 14658)


In [ ]:
# Generate matrix of word vectors
tfidf_matrix_nahj = vectorizer_nahj.fit_transform(df_nahj_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_nahj.shape)

(800, 21094)


In [ ]:
# Generate matrix of word vectors
tfidf_matrix_sahife = vectorizer_sahife.fit_transform(df_sahife_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_sahife.shape)

(924, 7711)


In [ ]:
# Generate matrix of word vectors
tfidf_matrix_quran_char = vectorizer_quran_char.fit_transform(df_verse_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_quran_char.shape)

(6236, 40)


In [ ]:
# Generate matrix of word vectors
tfidf_matrix_nahj_char = vectorizer_nahj_char.fit_transform(df_nahj_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_nahj_char.shape)

(800, 36)


In [ ]:
# Generate matrix of word vectors
tfidf_matrix_sahife_char = vectorizer_sahife_char.fit_transform(df_sahife_complete_dict_nrmlz['verse'])
# Print the shape of tfidf_matrix
print(tfidf_matrix_sahife_char.shape)

(924, 47)


In [ ]:
quran_indices_nrmlz = pd.Series(df_verse_complete_dict_nrmlz.index, index = df_verse_complete_dict_nrmlz['verse'])
nahj_indices_nrmlz = pd.Series(df_nahj_complete_dict_nrmlz.index, index = df_nahj_complete_dict_nrmlz['verse'])
sahife_indices_nrmlz = pd.Series(df_sahife_complete_dict_nrmlz.index, index = df_sahife_complete_dict_nrmlz['verse'])

In [ ]:
input = 'مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ'
input = normalize_arabic(input)
input = [input]

In [ ]:
def get_similars_from_quran(input, vectors, indices, mode='word'):

    if mode == 'word':
          tfidf_matrix_input = vectorizer_quran.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_quran)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_quran_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_quran_char)

    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_verse_complete_dict['verse'].iloc[indices]

In [ ]:
def get_similars_from_nahj(input, vectors, indices, mode = 'word'):

    if mode == 'word':
          tfidf_matrix_input = vectorizer_nahj.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_nahj)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_nahj_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_nahj_char)


    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_nahj_complete_dict['verse'].iloc[indices]

In [ ]:
def get_similars_from_sahife(input, vectors, indices, mode='word'):

    if mode == 'word':
          tfidf_matrix_input = vectorizer_sahife.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_sahife)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_sahife_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_sahife_char)

    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_sahife_complete_dict['verse'].iloc[indices]

In [ ]:
recoms = pd.DataFrame(get_similars_from_quran(input, vectorizer_quran , quran_indices_nrmlz, 'word'))
recoms

,verse
6233,مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6226,مِنْ شَرِّ مَا خَلَقَ
6228,وَمِنْ شَرِّ النَّفَّاثَاتِ فِي الْعُقَدِ
6227,وَمِنْ شَرِّ غَاسِقٍ إِذَا وَقَبَ
6229,وَمِنْ شَرِّ حَاسِدٍ إِذَا حَسَدَ
1214,إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ الَّذِي...
1181,إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ الصُّمّ...
6135,إِنَّ الَّذِينَ كَفَرُوا مِنْ أَهْلِ الْكِتَاب...
2888,الَّذِينَ يُحْشَرُونَ عَلَىٰ وُجُوهِهِمْ إِلَى...
5601,فَوَقَاهُمُ اللَّهُ شَرَّ ذَٰلِكَ الْيَوْمِ وَ...


In [ ]:
recoms = pd.DataFrame(get_similars_from_nahj(input, vectorizer_nahj , nahj_indices_nrmlz, 'word'))
recoms

,verse
557,وَ قَالَ ع المَرأَةُ شَرّ کُلّهَا وَ شَرّ مَا ...
798,وَ قَالَ ع شَرّ الإِخوَانِ مَن تُکُلّفَ لَهُ
351,وَ قَالَ ع فَاعِلُ الخَیرِ خَیرٌ مِنهُ وَ فَاع...
688,وَ قَالَ ع یأَتیِ عَلَی النّاسِ زَمَانٌ لَا یَ...
706,وَ قَالَ ع مَا خَیرٌ بِخَیرٍ بَعدَهُ النّارُ و...
25,إِنّ اللّهَ بَعَثَ مُحَمّداً ص نَذِیراً لِلعَا...
245,وَ إِنّ عَمَلَکَ لَیسَ لَکَ بِطُعمَةٍ وَ لَکِن...
267,فَاخفِض لَهُم جَنَاحَکَ وَ أَلِن لَهُم جَانِبَ...
57,أَصَابَکُم حَاصِبٌ وَ لَا بقَیِ‌َ مِنکُم آثِرٌ...
279,فَإِنّکَ قَد جَعَلتَ دِینَکَ تَبَعاً لِدُنیَا ...


In [ ]:
recoms = pd.DataFrame(get_similars_from_sahife(input, vectorizer_sahife , sahife_indices_nrmlz, 'word'))
recoms

,verse
583,وَ ارْحَمْ بِالْقُرْآنِ فِي مَوْقِفِ الْعَرْض...
650,السَّلَامُ عَلَیْکَ مِنْ شَهْرٍ هُوَ مِنْ کُل...
693,فَقَدْ ظَاهَرْتَ الْحُجَجَ، وَ أَبْلَیْتَ الْ...
274,إِنَّ ذَلِکَ لَا یَضِیقُ عَلَیْکَ فِي وُسْعِک...
392,اللَّهُمَّ أَعْطِنَا جَمِیعَ ذَلِکَ بِتَوْفِی...
379,اللَّهُمَّ صَلِّ عَلَی مُحَمَّدٍ وَ آلِه‌ِ، و...
701,وَ أَنْتَ اللَّه‌ُ لَا إِلَه‌َ إِلَّا أَنْتَ،...
661,اللَّهُمَّ فَلَکَ الْحَمْدُ إِقْرَاراً بِالْإ...
107,اللَّهُمَّ فَصَلِّ عَلَی مُحَمَّدٍ وَ آلِه‌ِ،...
78,فَصَلِّ عَلَیْهِمْ یَومَ یَأْتِي کُلُّ نَفْسٍ...


In [ ]:
recoms = pd.DataFrame(get_similars_from_quran(input, vectorizer_quran , quran_indices_nrmlz, 'char'))
recoms

,verse
6233,مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
50,أَتَأْمُرُونَ النَّاسَ بِالْبِرِّ وَتَنْسَوْنَ...
5037,أَأَنْتُمْ تَخْلُقُونَهُ أَمْ نَحْنُ الْخَالِق...
6177,إِنَّ الْإِنْسَانَ لَفِي خُسْرٍ
3912,أَتَدْعُونَ بَعْلًا وَتَذَرُونَ أَحْسَنَ الْخَ...
976,قَالَا رَبَّنَا ظَلَمْنَا أَنْفُسَنَا وَإِنْ ل...
3428,وَمِنْ آيَاتِهِ أَنْ خَلَقَكُمْ مِنْ تُرَابٍ ث...
2105,سُنَّةَ مَنْ قَدْ أَرْسَلْنَا قَبْلَكَ مِنْ رُ...
2128,قُلْ لَوْ أَنْتُمْ تَمْلِكُونَ خَزَائِنَ رَحْم...
4113,أَنْ تَقُولَ نَفْسٌ يَا حَسْرَتَا عَلَىٰ مَا ف...


In [ ]:
recoms = pd.DataFrame(get_similars_from_nahj(input, vectorizer_nahj , nahj_indices_nrmlz, 'char'))
recoms

,verse
255,أللّهُمّ إِلَیکَ أَفضَتِ القُلُوبُ وَ مُدّتِ ا...
52,وَ مِن تَمَامِ الأُضحِیّةِ استِشرَافُ أُذُنِهَ...
535,وَ قَالَ ع مَن نَالَ استَطَالَ
29,لَو أَمَرتُ بِهِ لَکُنتُ قَاتِلًا أَو نَهَیتُ ...
679,وَ قَالَ ع لَا تَظُنّنّ بِکَلِمَةٍ خَرَجَت مِن...
711,وَ قَالَ ع تَکَلّمُوا تُعرَفُوا فَإِنّ المَرءَ...
142,أَلَا وَ إِنّ الأَرضَ التّیِ تُقِلّکُم وَ السّ...
55,وَ لَقَد کُنّا مَعَ رَسُولِ اللّهِ ص نَقتُلُ آ...
628,وَ قَالَ ع اتّقُوا ظُنُونَ المُؤمِنِینَ فَإِنّ...
798,وَ قَالَ ع شَرّ الإِخوَانِ مَن تُکُلّفَ لَهُ


In [ ]:
recoms = pd.DataFrame(get_similars_from_sahife(input, vectorizer_sahife , sahife_indices_nrmlz, 'char'))
recoms

,verse
543,اللَّهُمَّ إِنِّي أَعْتَذِرُ إِلَیْکَ مِنْ مَ...
867,وَ کَمْ مِنْ حَاسِدٍ قَدْ شَرِقَ بِي بِغُصَّت...
316,اللَّهُمَّ وَ أَنْطِقْنِي بِالْهُدَی، وَ أَلْ...
204,وَ نَسَبْتَهُمْ إِلَی الْفَقْرِ وَ هُمْ أَهْل...
258,أَنْتَ الَّذِي وَصَفْتَ نَفْسَکَ بِالرَّحْمَة...
297,اللَّهُمَّ اسْقِنَا غَیْثاً مُغِیثاً مَرِیعاً...
343,اللَّهُمَّ صَلِّ عَلَی مُحَمَّدٍ وَ آلِه‌ِ، و...
304,اللَّهُمَّ صَلِّ عَلَی مُحَمَّدٍ وَ آلِه‌ِ، و...
877,اللَّهُمَّ إِنِّي وَجَدْتُ فِیمَا أَنْزَلْتَ ...
389,اللَّهُمَّ أَعْطِنِي کُلَّ سُؤْلِي، وَ اقْضِ ...
